## 1.10 从序列中移除重复项且保持元素间顺序不变
### 1.10.1 问题
去除序列中出现的重复的元素，但仍然保持剩下的元素顺序不变。
### 1.10.2 解决方案
如果序列中的值是可哈希(hashable)的，那么可以通过使用集合和生成器轻松解决。

如果一个对象是可哈希的，那么再它的生存期内必须是**不可变的**，它需要有一个`__hash__()`方法。
整数、浮点数、字符串、元组都是不可变的。

In [1]:
def dedupe(items):
    seen = set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

In [2]:
a = [1, 5, 2, 1, 9, 1, 5, 10]
list(dedupe(a))

[1, 5, 2, 9, 10]

只有当序列中的元素是可哈希的时候才能这么做。如果想在不可哈希的对象(比如列表)序列中去除重复项，需要对上述代码稍作修改：

In [10]:
def dedupe(items, key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)
    print(seen)

In [11]:
# 参数key的作用是指定一个函数用来将序列中的元素转换为可哈希的类型，这么做是为了检测重复项。
a = [{'x':1, 'y':2}, {'x':1, 'y':3}, {'x':1, 'y':2}, {'x':2, 'y':4}]

In [12]:
list(dedupe(a, key=lambda k: (k['x'], k['y']))) # 去除x, y都相同的元素

{(2, 4), (1, 2), (1, 3)}


[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 4}]

In [13]:
list(dedupe(a, key=lambda k: k['x']))   # 去除x值相同的元素

{1, 2}


[{'x': 1, 'y': 2}, {'x': 2, 'y': 4}]

### 1.10.3 讨论
如果想要做的只是去除重复项，最简单的方法就是构建一个集合。

但是这种方法不能保证元素间的顺序不变，因此得到的结果会被打乱。

In [14]:
a = [1, 5, 2, 1, 9, 1, 5, 10]
set(a)

{1, 2, 5, 9, 10}

## 1.11 对切片命名
### 1.11.1 问题与解决方案
假设有一些代码用来从字符串的固定位置中取出具体的数据。可以对切片命名，避免使用许多神秘难懂的硬编码索引。

In [16]:
record = '....................100.......513.25..........'
cost = int(record[20:23]) * float(record[30:36])
cost

51325.0

In [17]:
# 对切片命名
SHARES = slice(20, 23)
PRICE = slice(30, 36)
cost = int(record[SHARES]) * float(record[PRICE])
cost

51325.0

### 1.11.2 讨论
作为一条基本准则，代码中如果有很多硬编码的索引值，将导致可读性和可维护性都不佳。

一般来说，内置的`slice()`函数会创建一个切片对象，可以用在任何允许进行切片操作的地方。

In [18]:
items = [0, 1, 2, 3, 4, 5, 6]
a = slice(2, 4)
print(items[2:4])
print(items[a])

[2, 3]
[2, 3]


In [19]:
items[a] = [10, 11] # 通过索引改变元素的值
print(items)
del items[a]        # 通过索引删除元素
print(items)

[0, 1, 10, 11, 4, 5, 6]
[0, 1, 4, 5, 6]


In [20]:
# 如果有一个slice对象的实例s，可以通过`s.start, s.stop, s.step`属性来得到关于该对象的信息。
a = slice(10, 50, 2)
print(a.start)
print(a.stop)
print(a.step)

10
50
2


此外，可以通过使用`indices(size)`方法将切片映射到特定大小的序列上。返回一个(start, stop, step)元组，所有的值都已经恰当地限制在边界以内(当做索引操作时可避免出现`IndexError`异常)。 

In [32]:
a = slice(5, 12, 2)
s = 'HelloWorld'
a.indices(len(s))
a

slice(5, 12, 2)

In [33]:
for i in range(*a.indices(len(s))):
    print(s[i])

W
r
d


In [34]:
assert range(*a.indices(len(s))) == range(len(s))[a]

## 1.12 找出序列中出现次数最多的元素
### 1.12.2 解决方案
`collections`模块中的`Counter`类正是为此类问题所设计的。它有一个非常方便的`most_common()`方法可以直接得到答案。

如下有一个列表，列表中是一系列的单词，找出出现最频繁的单词。

In [35]:
words = [
    'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes',
    'the', 'eyes', 'the', 'eyes', 'the', 'eyes', 'not', 'around', 'the',
    'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into',
    'my', 'eyes', "you're", 'under'
]
from collections import Counter
word_counts = Counter(words)
top_three = word_counts.most_common(3)
print(top_three)

[('eyes', 8), ('the', 5), ('look', 4)]


### 1.12.3 讨论
可以给`Counter`对象提供任何可哈希的对象序列作为输入。在底层实现中，`Counter`是一个字典，在元素和它们出现的次数间做了映射。

In [36]:
word_counts['not']

1

In [37]:
word_counts['eyes']

8

In [38]:
# 如果想手动增加技术，只需要简单地自增即可：
morewords = ['why', 'are', 'not', 'looking', 'in', 'my', 'eyes']
for word in morewords:
    word_counts[word] += 1

word_counts['eyes']

9

In [39]:
# 另一种方法是使用update()
word_counts.update(morewords)
word_counts['eyes']

10

In [42]:
# `Counter`对象有一个特性，就是它们可以轻松地同各种数学运算操作结合起来使用。
a = Counter(words)
b = Counter(morewords)

In [43]:
a

Counter({'look': 4,
         'into': 3,
         'my': 3,
         'eyes': 8,
         'the': 5,
         'not': 1,
         'around': 2,
         "don't": 1,
         "you're": 1,
         'under': 1})

In [44]:
b

Counter({'why': 1,
         'are': 1,
         'not': 1,
         'looking': 1,
         'in': 1,
         'my': 1,
         'eyes': 1})

In [45]:
# Combine counts
c = a + b
c

Counter({'look': 4,
         'into': 3,
         'my': 4,
         'eyes': 9,
         'the': 5,
         'not': 2,
         'around': 2,
         "don't": 1,
         "you're": 1,
         'under': 1,
         'why': 1,
         'are': 1,
         'looking': 1,
         'in': 1})

In [46]:
# Subtract counts
d = a - b
d

Counter({'look': 4,
         'into': 3,
         'my': 2,
         'eyes': 7,
         'the': 5,
         'around': 2,
         "don't": 1,
         "you're": 1,
         'under': 1})

## 1.13 通过公共键对字典列表排序
### 1.13.1 问题
有一个字典列表，想根据一个或多个字典中地值来对列表排序。
### 1.13.2 解决方案
利用`operator`模块中地`itemgetter`函数对这类结构进行排序非常简单。

假设通过查询数据库表项获取网站上地成员列表，得到如下地数据结构：

In [47]:
rows = [
    {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
    {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
    {'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
    {'fname': 'Big', 'lname': 'Jones', 'uid': 1004},
]

In [48]:
from operator import itemgetter
rows_by_fname = sorted(rows, key=itemgetter('fname'))
rows_by_uid = sorted(rows, key=itemgetter('uid'))
print(rows_by_fname)
print(rows_by_uid)

[{'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}]
[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}]


In [49]:
# itemgetter()函数可以接受多个键
rows_by_lfname = sorted(rows, key=itemgetter('lname', 'fname'))
print(rows_by_lfname)

[{'fname': 'David', 'lname': 'Beazley', 'uid': 1002}, {'fname': 'John', 'lname': 'Cleese', 'uid': 1001}, {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}, {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003}]


In [51]:
# 使用匿名函数也可以实现类似功能
rows_uid = sorted(rows, key=lambda k:k['uid'])
rows_uid

[{'fname': 'John', 'lname': 'Cleese', 'uid': 1001},
 {'fname': 'David', 'lname': 'Beazley', 'uid': 1002},
 {'fname': 'Brian', 'lname': 'Jones', 'uid': 1003},
 {'fname': 'Big', 'lname': 'Jones', 'uid': 1004}]

### 1.13.3 讨论
上述代码中，rows被传递给内建地`sorted()`函数，该函数接受一个关键字参数`key`。这个参数应该代表一个可调用对象(callable)，该对象从rows中接受一个单独的元素作为输入并返回一个用来做排序依据的值。

`itemgetter()`函数创建的就是这样一个可调用对象。函数`operator.itemgetter()`接受的参数可作为查询的标记，用来从rows的记录中提取出所需要的值。它可以是字典的键名称、用数字表示的列表元素或是任何可以传给对象的`__getitem__()`方法的值。如果传多个标记给`itemgetter()`，那么它产生的可调用对象将返回一个包含所有元素在内的元组，然后`sorted()`将根据对元组的排序结果来排列输出结果。

## 1.14 对不原生支持的比较操作的对象排序
### 1.14.1 问题
在同一个类的实例之间做排序，但是它们并不原生支持比较操作。
### 1.14.2 解决方案
内建的`sorted()`函数可接受一个用来传递可调用对象(callable)的参数key，而该可调用对象会返回待排序对象中的某些值，sorted则利用这些值来比较对象。

例如，如果应用中有一系列的User对象实例，欲通过`user_id`属性来对它们排序，则可以提供一个可调用对象将User实例作为输入然后返回`user_id`。示例如下：

In [1]:
class User:
    def __init__(self, user_id):
        self.user_id = user_id
    def __repr__(self):
        return 'User({})'.format(self.user_id)


users = [User(23), User(3), User(99)]
users 

[User(23), User(3), User(99)]

In [3]:
sorted(users, key=lambda u: u.user_id)

[User(3), User(23), User(99)]

In [4]:
# 除了可以用`lambda`表达式外，另一种方式是使用operator.attrgetter()
from operator import attrgetter
sorted(users, key=attrgetter('user_id'))

[User(3), User(23), User(99)]

### 1.14.3 讨论
要使用`lambda`表达式还是`attrgetter()`或许只是一种个人喜好。通常，`attrgetter()`要更快一些，而且具有允许同时提取多个字段值的能力。这和针对字典的`operator.itemgetter()`的使用很类似(参见1.13节)。例如，User实例还有一个`first_name`和`last_name`属性的话，可以执行如下的排序操作：

```python
by_name = sorted(users, key=attrgetter('last_name', 'first_name'))
```
同时，本节所用到的奇数，也适用于像`min()`和`max()`这样的函数。例如：

In [5]:
min(users, key=attrgetter('user_id'))

User(3)

## 1.15 根据字段将记录分组
### 1.15.1 问题
有一系列的字典或对象实例，向根据某个特定的字段来分组迭代数据。
### 1.15.2 解决问题
`itertools.groupby()`函数在对数据进行分组时特别有用。

为了说明其用途，假设有如下的字典列表，现在假设想根据日期以分组的方式迭代数据。首先以目标字段来对序列排序，然后再使用`itertools.groupby()`

In [8]:
rows = [
    {'address': '5412 N CLARK', 'date': '07/01/2012'},
    {'address': '5148 N CLARK', 'date': '07/04/2012'},
    {'address': '5800 E 58TH', 'date': '07/02/2012'},
    {'address': '2122 N CLARK', 'date': '07/03/2012'},
    {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
    {'address': '1060 W ADDISON', 'date': '07/02/2012'},
    {'address': '4801 N BROADWAY', 'date': '07/01/2012'},
    {'address': '1039 W GRANVILLE', 'date': '07/04/2012'},
]

from operator import itemgetter
from itertools import groupby

# Sort by the desired field first
rows.sort(key=itemgetter('date'))
# Iterate in groups
for date, items in groupby(rows, key=itemgetter('date')):
    print(date)
    for i in items:
        print(' ', i)

07/01/2012
  {'address': '5412 N CLARK', 'date': '07/01/2012'}
  {'address': '4801 N BROADWAY', 'date': '07/01/2012'}
07/02/2012
  {'address': '5800 E 58TH', 'date': '07/02/2012'}
  {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'}
  {'address': '1060 W ADDISON', 'date': '07/02/2012'}
07/03/2012
  {'address': '2122 N CLARK', 'date': '07/03/2012'}
07/04/2012
  {'address': '5148 N CLARK', 'date': '07/04/2012'}
  {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}


### 1.15.3 讨论
函数`groupby()`通过扫描序列找出拥有相同值(或是由参数key指定的函数所返回的值)的序列项，并将它们分组。`groupby()`创建了一个迭代器，而再每次迭代时都会返回一个值(value)和一个子迭代器(sub_iterator)，这个子迭代器可以产生所有在该分组内具有该值的项。

这里重要的是首先要根据感兴趣的字段对数据进行排序。因为`groupby()`**只能检查连续的项**，不首先排序的话，将无法按所想的方式来对记录分组。

如果只是简单地根据日期将数据分组到一起，放在一个大的数据结构中以允许进行随机访问，可以利用`defaultdict()`构建一个一键多值字典(multidict，见1.6节)。不需要先对记录做排序。

In [9]:
from collections import defaultdict
rows_by_date = defaultdict(list)
for row in rows:
    rows_by_date[row['date']].append(row)

# 这样可以轻松地访问每个日期地记录
for r in rows_by_date['07/01/2012']:
    print(r)

{'address': '5412 N CLARK', 'date': '07/01/2012'}
{'address': '4801 N BROADWAY', 'date': '07/01/2012'}


In [10]:
rows_by_date

defaultdict(list,
            {'07/01/2012': [{'address': '5412 N CLARK', 'date': '07/01/2012'},
              {'address': '4801 N BROADWAY', 'date': '07/01/2012'}],
             '07/02/2012': [{'address': '5800 E 58TH', 'date': '07/02/2012'},
              {'address': '5645 N RAVENSWOOD', 'date': '07/02/2012'},
              {'address': '1060 W ADDISON', 'date': '07/02/2012'}],
             '07/03/2012': [{'address': '2122 N CLARK', 'date': '07/03/2012'}],
             '07/04/2012': [{'address': '5148 N CLARK', 'date': '07/04/2012'},
              {'address': '1039 W GRANVILLE', 'date': '07/04/2012'}]})

## 1.16 筛选序列中的元素
### 1.16.1 问题
序列中含有一些数据，需要提取其中的值或根据某些标准对学列做删减。
### 1.16.2 解决方案
筛选学列中的数据，通常最简单的方法是使用**列表推导式**(list comprehension)。

In [11]:
mylist = [1, 4, -5, 10, -7, 2, 3, -1]
# 筛选出大于0的元素
[n for n in mylist if n > 0]

[1, 4, 10, 2, 3]

使用列表推导式的一个潜在缺点是如果原始输入非常大的话，这么做可能会产生一个庞大的结果。可以**使用生成器表达式通过迭代的方式产生筛选的结果**。例如：

In [12]:
pos = (n for n in mylist if n > 0)
pos

<generator object <genexpr> at 0x000001FBBB377200>

In [13]:
for x in pos:
    print(x)

1
4
10
2
3


有时候筛选的标准没法简单地表示在列表推导式或生成器表达式中。比如，假设筛选过程涉及异常处理或者其他一些复杂的细节。基于此，可以将处理筛选逻辑的代码放到单独的函数，然后使用内建的`filter()`函数处理。

`filter()`创建了一个迭代器，如果想要的是列表形式的结果，确保加上了`list()`。

In [14]:
values = ['1', '2', '-3', '-', '4', 'N/A', '5']

def is_int(val):
    try:
        x = int(val)
        return True
    except ValueError:
        return False

ivals = list(filter(is_int, values))
print(ivals)

['1', '2', '-3', '4', '5']


### 1.16.3 讨论
列表推导式和生成器表达式通常是用来筛选数据的最简单和最直接的方式。此外，它们也具有同时对数据做转换的能力。

In [15]:
mylist = [1,4,-5,10,-7,2,3,-1]
import math
[math.sqrt(n) for n in mylist if n > 0]

[1.0, 2.0, 3.1622776601683795, 1.4142135623730951, 1.7320508075688772]

关于筛选数据，有一种情况是用新值替换掉不满足标准的值，而不是丢弃它们。例如，除了要找到的正整数之外，我们也许还希望在指定的范围内将不满足要求的值替换掉。通常，这可以**将筛选条件移到一个条件表达式中**来轻松实现。

In [16]:
clip_neg = [n if n > 0 else 0 for n in mylist]
clip_neg

[1, 4, 0, 10, 0, 2, 3, 0]

In [17]:
clip_pos = [n if n < 0 else 0 for n in mylist]
clip_pos

[0, 0, -5, 0, -7, 0, 0, -1]

`itertools.compress()`接受一个可迭代对象以及一个布尔选择器序列作为输入。输出时，它会给出所有在相应的布尔选择器中为True的可迭代对象元素。如果想把对一个序列的筛选结果施加到另一个相关的序列上时，这就会非常有用。

In [18]:
address = [
    '5412 N CLARK',
    '5148 N CLARK',
    '5800 E 58TH',
    '2122 N CLARK',
    '5645 N RAVENSWOOD',
    '1060 W ADDISON',
    '4801 N BROADWAY',
    '1039 W GRANVILLE',
]

counts = [0, 3, 10, 4, 1, 7, 6, 1]

想构建一个地址列表，其中相应的counts值要大于5。

这里的关键在于首先创建一个布尔序列，用来表示哪个元素可满足我们的条件。然后`compress()`函数挑选出满足布尔值为True的相应元素。

同 `filter()`函数一样，正常情况下`compress()`会返回一个迭代器。一次，如果需要的话，得使用list()将结果转化为列表。

In [19]:
from itertools import compress
more5 = [n > 5 for n in counts]
more5

[False, False, True, False, False, True, True, False]

In [20]:
list(compress(address, more5))

['5800 E 58TH', '1060 W ADDISON', '4801 N BROADWAY']

## 1.17 从字典中提取子集
### 1.17.1 问题
想创建一个字典，其本身是另一个字典的子集。
### 1.17.2 解决方案
利用字典推导式(dictionary comprehension)可轻松解决。例如：

In [22]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}
# Make a dictionary of all prices over 200
p1 = {key:value for key, value in prices.items() if value > 200}

# Make a dictionary of tech stocks
tech_names = {'AAPL', 'IBM', 'HPQ', 'MSFT'}
p2 = {key:value for key, value in prices.items() if key in tech_names}

### 1.17.3 讨论
大部分可以用字典推导式解决的问题也可以通过创建元组序列然后将它们传给`dict()`函数来完成。例如：
```python
p1 = dict((key,value) for key, value in prices.items() if value > 200)
```
但是字典推导式的方案更加清晰，而且实际运行起来也要快很多。

有时候会有多种方法来完成同一件事情。例如，第二个例子还可以重写。但是，即使测试表明这种解决方案比第一种慢上1.6倍。

In [23]:
# Make a dictionary of tech stocks
tech_names = {'AAPL', 'IBM', 'HPQ', 'MSFT'}
p2 = {key: prices[key] for key in prices.keys() & tech_names}   # 取交集

## 1.18 将名称映射到序列的元素中
### 1.18.1 问题
我们的代码时通过位置(即索引，或下标)来访问列表或元组的，但有时候会使代码变得有些难以阅读。我们希望可以**通过名称来访问元素，以此减少结构中对位置的依赖性**。
### 1.18.2 解决方案
相比普通的元组，`collections.namedtuple()`(命名元组)只增加了极小的开销就提供了这些便利。它返回的是Python中标准元组类型的子类。我们提供给它一个类型名称以及相应的字段，它就会返回一个可实例化的类、为你已经定义好的字段传入值等。例如：

In [25]:
from collections import namedtuple
Subscriber = namedtuple('Subscriber', ['addr', 'joined'])
sub = Subscriber('charlie@example.com', '1999-12-20')
sub

Subscriber(addr='charlie@example.com', joined='1999-12-20')

In [26]:
# docstring for the new class
Subscriber.__doc__

'Subscriber(addr, joined)'

尽管`namedtuple`的实例看起来就像一个普通的类实例，但它的实例与普通的元组是可互换的，而且支持所有普通的元组所支持的操作，例如索引(indexing)和分解(unpacking)。

In [27]:
# sub['addr'], sub['joined']
sub.addr, sub.joined

('charlie@example.com', '1999-12-20')

In [28]:
len(sub)    # 2
addr, joined = sub  # ...

命名元组的主要作用在于**将代码同它所控制的元素位置间解耦**。所以，如果从数据库调用中得到一个大型的元组列表，而且通过元素的位置来访问数据，那么假如在表单中新增一列数据，那么代码就会崩溃。但如果首先将返回的元组转型为命名怨怒，就不会出现问题。

In [29]:
def compute_cost(records):
    total = 0.0
    for rec in records:
        total += rec[1] * rec[2]
    return total

# 通过位置来引用元素常常使得代码的表达力不够强，而且也很依赖记录的具体结构。下面是使用命名元组的版本：
from collections import namedtuple
Stock = namedtuple('Stock', ['name', 'shares', 'price'])
def compute_cost2(records):
    total = 0.0
    for rec in records:
        s = Stock(*rec)
        total += s.shares * s.price
    return total
# 如果records中的元素是某个类的实例，且已经有了shares和price这样的属性，那就可以直接通过属性来访问，不需要通过位置来引用，也就没有必要再转换成命名元组了。

### 1.18.3 讨论
`namedtuple`的一种可能用法是**作为字典的替代**，后者需要更多的空间来存储。因此，如果要构建涉及字典的大型数据结构，使用`namedtuple`更加高效。但是，与字典不同的是，`namedtuple`是**不可变的**(immutable)。例如：

In [30]:
s = Stock('ACME', 100, 123.45)
s

Stock(name='ACME', shares=100, price=123.45)

In [31]:
s.shares = 75

AttributeError: can't set attribute

如果要修改任何属性，可以通过使用`namedtuple`实例中的`_replace()`方法来实现。该方法会**创建一个全新的命名元组，并对相应的值做替换**。

In [32]:
s = s._replace(shares=75)
s

Stock(name='ACME', shares=75, price=123.45)

`_replace()`方法有一个微妙的用途，就是它可以作为一种简便的方法填充具有可选或缺失字段的命名元组。

首先创建一个包含默认值的“原型”元组，然后使用`_replace()`方法创建一个新的实例，把相应的值替换掉。

In [33]:
from collections import namedtuple
Stock = namedtuple('Stock', ['name', 'shares', 'price', 'date', 'time'])
# Create a prototype instance
stock_prototype = Stock('', 0, 0.0, None, None)
# Function to convert a dictionary to a Stock
def dict_to_stock(s):
    return stock_prototype._replace(**s)

In [34]:
a = {'name': 'ACME', 'shares': 100, 'price': 123.45}
dict_to_stock(a)

Stock(name='ACME', shares=100, price=123.45, date=None, time=None)

In [35]:
b = {'name': 'ACME', 'shares': 100, 'price': 123.45, 'date':'12/17/2012'}
dict_to_stock(b)

Stock(name='ACME', shares=100, price=123.45, date='12/17/2012', time=None)

## 1.19 同时对数据做转换和换算
### 1.19.1 问题
需要调用一个换算(reduction)函数(例如sum(), min(), max())，但首先得对数据做转换或筛选。
### 1.19.2 解决方案
有一种非常优雅的方式能将数据换算和转换结合在一起——在函数参数中使用生成器表达式。

例如，如果想计算平方，可以像下面这样：


In [38]:
nums = [1, 2, 3, 4, 5]
s = sum(x * x for x in nums)
s

55

In [39]:
#Determine if any .py files exist in a directory
import os
files = os.listdir('dirname')
if any(name.endswith('.py') for name in files):
    print('There by python!')
else:
    print('Sorry, no python.')

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'dirname'

In [40]:
# Output a tuple as CSV
s = ('ACME', 50, 12345)
print(','.join(str(x) for x in s))

ACME,50,12345


In [41]:
# Data reduction across fields of a data structure
portfolio = [
    {'name': 'GOOG', 'shares': 50},
    {'name': 'YHOO', 'shares': 75},
    {'name': 'AOL', 'shares': 20},
    {'name': 'SCOX', 'shares':65}
]
min_shares = min(s['shares'] for s in portfolio)
min_shares

20

### 1.19.3 讨论
这种解决方案展示了把生成器表达式作为函数的单独参数时在语法上的一些微妙之处(即，不必重复使用括号)。比如，下面这两行代码表示的是同一个意思：

```python
s = sum((x * x for x in nums))
s = sum(x * x for x in nums)
```

比起首先创建一个临时的列表，使用生成器做参数通常是更为高效和优雅的方式。如果nums非常巨大，那么就会创建一个庞大的临时数据结构。基于生成器的解决方案可以以迭代的方式转换数据，因此在内存使用上高效更多。
```python
nums = [1, 2, 3, 4, 5]
s = sum([x * x for x in nums])
```
某些特定的换算函数比如`min()`和`max()`都可接受一个key参数，当可能倾向于使用生成器时会很有帮助。比如在portfolio的例子中，也许会考虑下面这种替代方案：

In [42]:
# Original: returns 20
min_shares = min(s['shares'] for s in portfolio)
# Alternative: returns {'name': 'AOL', 'shares': 20}
min_shares = min(portfolio, key=lambda s: s['shares'])

## 1.20 将多个映射合并为单个映射
### 1.20.1 问题
有多个字典或映射，想在逻辑上将它们合并为一个单独的映射结构，以此执行某些特定的操作，比如查找值或检查键是否存在。
### 1.20.2 解决方案
假设有两个字典：
```python
a = {'x': 1, 'z': 3}
b = {'y': 2, 'z': 4}
```
现在假设想执行查找操作，必须得检查这两个字典(例如，先在a中查找，如果没找到再去b中查找)。一种简单的方法是利用`collections`模块中的`ChainMap`类来解决这个问题。例如：

In [43]:
a = {'x': 1, 'z': 3}
b = {'y': 2, 'z': 4}

from collections import ChainMap
c = ChainMap(a, b)
print(c['x'])   # Outputs 1 (from a)
print(c['y'])   # Outputs 2 (from b)
print(c['z'])   # Outputs 3 (from a)

1
2
3


### 1.20.3 讨论
`ChinaMap`**可接受多个映射然后在逻辑上使它们表现为一个单独的映射结构**。但是，这些映射在字面上并不会合并在一起。相反，`ChainMap`只是简单地维护一个记录底层映射关系的列表，然后重定义常见的字典操作来扫描这个列表。大部分的操作都能正常工作。

In [44]:
len(c) # 3
list(c.keys()) # ['x', 'y', 'z']
list(c.values()) # [1, 2, 3]

[2, 3, 1]

**如果有重复的键，那么这里会采用第一个映射中所对应的值**。

**修改映射的操作总是会作用在列出的第一个映射结构上**。例如：

In [45]:
c['z'] = 10
c['w'] = 40
del c['x']
a

{'z': 10, 'w': 40}

In [47]:
del c['y']  # 在第一个映射中找不到键'y'

KeyError: "Key not found in the first mapping: 'y'"

`ChainMap`与带有作用域的值，比如编程语言中的变量(即全局变量、局部变量等)一起工作时特别有用。

In [48]:
values = ChainMap()
values['x'] = 1
# Add a new mapping
values = values.new_child()
values['x'] = 2
# Add a new mapping
values = values.new_child()
values['x'] = 3
values

ChainMap({'x': 3}, {'x': 2}, {'x': 1})

In [49]:
# Discard last mapping
values = values.parents
values['x']

2

In [50]:
# Discard last mapping
values = values.parents
values['x']

1

In [51]:
values

ChainMap({'x': 1})

作为`ChainMap`的替代方案，可能会考虑利用字典的`update()`方法将多个字典合并在一起。

In [52]:
a = {'x': 1, 'z': 3}
b = {'y': 2, 'z': 4}
merged = dict(b)
merged.update(a)
merged['x']

1

In [53]:
merged['y']

2

In [54]:
merged['z']

3

这样做行得通，但这需要单独构建一个完整的字典对象(或者修改其中现有的一个字典，这就破坏了原始数据)。此外，如果其中任何一个原始字典做了修改，这个改变都不会反应到合并后的字典中。例如：

In [55]:
a['x'] = 13
merged['x']

1

`ChainMap`使用的就是原始的字典，因此不会产生这种令人不悦的行为。

In [56]:
a = {'x': 1, 'z': 3}
b = {'y': 2, 'z': 4}
merged = ChainMap(a, b)
merged['x']

1

In [57]:
a['x'] = 56
merged['x']

56